# Manipulation

## CSV

### Imports

In [1]:
import csv

### Functions

#### Read

In [2]:
def csv_read_file(path):
    ''' Lê o arquivo csv e retorna seus dados em uma lista '''
    data_file = []
    with open(path, 'r', encoding='utf-8') as file:
        read = csv.reader(file)
        for row in read:
            data_file.append(row)
    return data_file


#### Write

In [3]:
def csv_write_file(path, collection):
    ''' Escreve os dados de uma coleção no arquivo CSV linha a linha '''
    with open(path, 'w', encoding='utf-8', newline='') as file:
        write = csv.writer(file)
        for row in collection:
            write.writerow(row)
    return 'File written with success!'

#### Create file CSV

- Para testar as funções, criei uma arquivo CSV com alguns nomes aleatórios que peguei no site https://rogertakemiya.com.br/gerador-de-nomes-de-pessoas/ "Lista com alguns dos nomes utilizados no Estados Unidos".

In [4]:
import random

delivery = ['2022-02-12','2022-02-16','2022-02-08','2022-02-22','2022-02-28']
register = ['2022-02-02','2022-02-05','2022-01-23','2022-02-01','2022-01-28']
type_order = ['design','paisagem','logo','personagem','caricatura']

data = csv_read_file("../Data/test_copy.csv")
for row in data[1:]:
    row[1] = random.choice(type_order)
    row.append(random.choice(delivery))
    row.append(random.choice(register))
csv_write_file('../Data/final_test.csv', data)

'File written with success!'

- Como tinha apenas os nomes, decidi criar uma lista com 5 possibilidades de datas de entrega, datas de registro, e tipos de pedidos de artisticos para poder popular o CSV com o que iria precisar.

## SQLite3

### Imports

In [5]:
import sqlite3
import os
from datetime import datetime

### Functions

#### Dates

In [6]:
def date_now():
    date_now = datetime.today().strftime('%Y-%m-%d')
    return str(date_now)

def date_to_display(date):
    date = date.split('-')[::-1]
    date = '/'.join(date)
    return date

def date_to_sqlite(date):
    date = date.split('/')[::-1]
    date = '-'.join(date)
    return date
    

#### Create and connect Database

In [7]:
connection = sqlite3.connect('../Data/DB_Orders.db')
cursor = connection.cursor()

def close_db():
    cursor.close()
    connection.close()

#### Create table

In [8]:
sql_create = """CREATE TABLE IF NOT EXISTS Orders 
(id INTEGER PRIMARY KEY AUTOINCREMENT, 
client TEXT, 
type_order TEXT, 
date_delivery TEXT, 
date_register TEXT NOT NULL)"""

def fundb_create_table():
    cursor.execute(sql_create)
    connection.commit()

#### Insert 

In [9]:
sql_insert = """INSERT INTO Orders (client, type_order, date_delivery, date_register) 
VALUES (?,?,?,?)"""

def fundb_insert_collection(collection):
    for row in collection:
        cursor.execute(sql_insert, row)
        connection.commit()
    
def fundb_insert_one(client, type_order, date_delivery):
    date_register = date_to_sqlite(date_now())
    cursor.execute(sql_insert, (client, type_order, date_delivery, date_register))
    connection.commit()

#### Recover

In [10]:
sql_select = """SELECT * FROM Orders"""

def fundb_recover_all():
    cursor.execute(sql_select)
    return cursor.fetchall()

def fundb_recover_specific(colum, specification):
    sql_where = sql_select + f" WHERE {colum} = '{specification}'"
    cursor.execute(sql_where)
    record = cursor.fetchall()
    return record    

def fundb_find_id(name_client):
    records = fundb_recover_specific('client', name_client)
    list_ids = []
    for rec in records:
        list_ids.append(rec[0])
    if len(list_ids) == 1:
        return list_ids[0]
    else:
        return list_ids

- Ao inves de fazer uma função para cada campo da coluna, fiz uma função que recebe a coluna que será usada como filtro e resgata todos os valores relacionados ao valor especificado.

#### Update

In [11]:
sql_update = "UPDATE Orders SET "

def fundb_update_by_id(order_id, column, new_data):
    sql_specific = sql_update + f"{column} = '{new_data}' WHERE id = {order_id}"
    cursor.execute(sql_specific)
    connection.commit()
    return f'Update OK!'

#### Delete

In [12]:

def fundb_delete_by_id(order_id):
    sql_delete = f"DELETE FROM Orders WHERE id = {order_id}"
    cursor.execute(sql_delete)
    connection.commit()
    return f'Record deleted'

# Runing

## Useful

In [13]:
def split_rec(rec):
    order_id = rec[0]
    client = rec[1]
    type_order = rec[2]
    delivery = date_to_display(rec[3])
    register = date_to_display(rec[4])
    print_recover(order_id, type_order, client, delivery, register)

def print_recover(order_id, type_order, client, delivery, register):
    display = '\n\n=============================\n'\
    f'ID: {order_id} \n'\
    f'Cliente: {client}\n'\
    f'Tipo do Pedido: {type_order}\n'\
    f'Data de Entrega: {delivery}\n'\
    f'Date de Registro: {register}\n\n'
    print(display)


### Displays

In [14]:
display_menu = """
=========== Menu ============

0 - Sair
1 - Mostrar pedidos
2 - Inserir novo pedido
3 - Atualizar um novo pedido
4 - Apagar um pedido

"""
display_op1 = """
=========== Menu ============

1 - Mostrar todos os pedidos
2 - Pedido específico

"""
display_inserir = """
=========== Menu ============

1 - Inserir: 
"""


### Options

In [15]:
def option_1():
    print(display_op1)
    option = int(input("Opção:  "))
    if option == 1:
        for rec in fundb_recover_all():
            split_rec(rec)
    elif option == 2:
        column = str(input("Buscar por: "))
        specific = str(input("Dado: "))
        for rec in fundb_recover_specific(column, specific):
            split_rec(rec)

def option_2():
    print(display_inserir)
    client = str(input("Cliente: "))
    type_order = str(input("Tipo do pedido: "))
    delivery = str(input("Data de entrega: "))
    register = date_now()
    fundb_insert_one(client, type_order, delivery)

def option_3():
    print(display_inserir)
    order_id = int(input("ID de quem quer atualizar: "))
    column = str(input("Campo a ser mudado: "))
    new_date = str(input("Atualização: "))
    if column == 'date_delivery' or columns == 'date_register':
        date_to_sqlite(new_date)
    fundb_update_by_id(order_id, column, new_date)

def option_4():
    print(display_inserir)
    order_id = int(input("ID de quem quer atualizar: "))
    ok = fundb_delete_by_id(order_id)
    return ok

def print_display_and_option(display):
    while True:
        print(display)
        option = int(input("Opção:  "))
        if option == 0:
            break
        elif option == 1:
            option_1()
            continue
        elif option == 2:
            option_2()
            continue
        elif option == 3:
            option_3()
            continue
        elif option == 4:
            option_4()
            continue
        else:
            print("Opção invalida")
            continue

## Run

### Reset Database

In [16]:
op = int(input("Reset Database? \n1 - Sim \n2 - Não \n"))
if op == 1:
    if os.path.exists('../Data/DB_Orders.db'):
        close_db()
        os.remove('../Data/DB_Orders.db')
    csv_path = '../Data/final_test.csv'
    data = csv_read_file(csv_path)[1:]
    connection = sqlite3.connect('../Data/DB_Orders.db')
    cursor = connection.cursor()
    fundb_create_table()
    fundb_insert_collection(data)

Reset Database? 
1 - Sim 
2 - Não 
 2


In [ ]:
print_display_and_option(display_menu)


=========== Menu ============

0 - Sair
1 - Mostrar pedidos
2 - Inserir novo pedido
3 - Atualizar um novo pedido
4 - Apagar um pedido




Opção:   1



=========== Menu ============

1 - Mostrar todos os pedidos
2 - Pedido específico




Opção:   2
Buscar por:  id
Dado:  50




ID: 50 
Cliente: Mr Clement Fleisher
Tipo do Pedido: design
Data de Entrega: 28/02/2022
Date de Registro: 23/01/2022



=========== Menu ============

0 - Sair
1 - Mostrar pedidos
2 - Inserir novo pedido
3 - Atualizar um novo pedido
4 - Apagar um pedido




...


=========== Menu ============

0 - Sair
1 - Mostrar pedidos
2 - Inserir novo pedido
3 - Atualizar um novo pedido
4 - Apagar um pedido




Opção:   1



=========== Menu ============

1 - Mostrar todos os pedidos
2 - Pedido específico




Opção:   1




ID: 1 
Cliente: Mr Pratik Skaggs
Tipo do Pedido: design
Data de Entrega: 28/02/2022
Date de Registro: 23/01/2022




ID: 2 
Cliente: Mr Uehudah Hack
Tipo do Pedido: logo
Data de Entrega: 08/02/2022
Date de Registro: 23/01/2022




ID: 3 
Cliente: Mr Edison Drye
Tipo do Pedido: personagem
Data de Entrega: 16/02/2022
Date de Registro: 01/02/2022




ID: 4 
Cliente: Ms Mihoko Scholl
Tipo do Pedido: paisagem
Data de Entrega: 22/02/2022
Date de Registro: 23/01/2022




ID: 5 
Cliente: Mr Ajani Harding
Tipo do Pedido: logo
Data de Entrega: 12/02/2022
Date de Registro: 01/02/2022




ID: 6 
Cliente: Ms Ruthie Coco
Tipo do Pedido: paisagem
Data de Entrega: 22/02/2022
Date de Registro: 23/01/2022




ID: 7 
Cliente: Mr Orazio Hart
Tipo do Pedido: personagem
Data de Entrega: 08/02/2022
Date de Registro: 05/02/2022




ID: 8 
Cliente: Ms Vinaya Justus
Tipo do Pedido: logo
Data de Entrega: 08/02/2022
Date de Registro: 01/02/2022




ID: 9 
Cliente: Mr Tristan Karns
Tipo do Pedido: paisagem
Data 